# 작사가 인공지능 만들기

In [1]:
#데이터 불러오기
import glob
import os
import re
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['"Don\'t worry about a thing,', "'Cause every little thing gonna be all right.", 'Singin\': "Don\'t worry about a thing,']


In [2]:
# 문장 indexing
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 스킵

    if idx > 9: break
        
    print(sentence)

"Don't worry about a thing,
'Cause every little thing gonna be all right.
Singin': "Don't worry about a thing,
'Cause every little thing gonna be all right!" Rise up this mornin',
Smiled with the risin' sun,
Three little birds
Perch by my doorstep
Singin' sweet songs
Of melodies pure and true,
Sayin', ("This is my message to you-ou-ou:") Singin': "Don't worry 'bout a thing,


In [3]:
# 문장 전처리 함수
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 소문자로 바꾸고 양쪽 공백을 삭제

    # 정규식을 이용하여 문장 처리
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence) # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'
    return sentence

In [4]:
corpus = []

# 모든 문장에 전처리 함수 적용
for sentence in raw_corpus:
    if len(sentence) == 0:
        continue
    if sentence[-1] == ":":
        continue
    temp = preprocess_sentence(sentence)
    if len(temp.split()) <= 15:
        corpus.append(temp)
        
print(corpus[:2])
print(len(corpus))

['<start> don t worry about a thing , <end>', '<start> cause every little thing gonna be all right . <end>']
156013


In [5]:
# 처리한 문장 확인
print(corpus[0].split())
len(corpus[0].split())

['<start>', 'don', 't', 'worry', 'about', 'a', 'thing', ',', '<end>']


9

In [6]:
# 토큰화
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=15000,  # 전체 단어의 개수 
        filters=' ',      # 전처리 로직
        oov_token="<unk>" # out-of-vocabulary, 사전에 없는 단어
    )
    tokenizer.fit_on_texts(corpus) # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축

    # tokenizer를 활용하여 모델에 입력할 데이터셋을 구축
    tensor = tokenizer.texts_to_sequences(corpus) # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding 메소드
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰진다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                           padding='post',
                                                           maxlen=15)

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   40   16 ...    0    0    0]
 [   2   66  124 ...    0    0    0]
 [   2 1544   40 ...    0    0    0]
 ...
 [   2   38  905 ...    0    0    0]
 [   2   38   68 ...    0    0    0]
 [   2    8   83 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f55a8ecc090>


In [7]:
print(len(tensor[10,:])) # 생성된 텐서 데이터 확인

15


In [8]:
# 단어 사전의 index
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 20: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to
11 : it
12 : me
13 : my
14 : in
15 : that
16 : t
17 : s
18 : on
19 : your
20 : of


In [9]:
# 문장 생성
src_input = tensor[:, :-1] # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높다.
tgt_input = tensor[:, 1:]  # tensor에서 <start>를 잘라내서 타겟 문장을 생성.

In [10]:
# 생성된 문장 확인
print(src_input[0])
print(tgt_input[0])

[  2  40  16 700 111   9 183   5   3   0   0   0   0   0]
[ 40  16 700 111   9 183   5   3   0   0   0   0   0   0]


In [11]:
# train, test set 분리
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                          tgt_input,
                                                          test_size=0.2,
                                                          shuffle=True)

In [12]:
# 분리된 데이터 확인
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124810, 14)
Target Train: (124810, 14)


In [13]:
# 데이터셋 구축
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 512
steps_per_epoch = len(enc_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # 0:<pad>를 포함하여 dictionary 갯수 + 1

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((512, 14), (512, 14)), types: (tf.int32, tf.int32)>

In [14]:
# 모델 생성 함수
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out 

In [15]:
# 모델 생성
embedding_size = 256 # 워드 벡터의 차원 수
hidden_size = 1024 # LSTM Layer의 hidden 차원 수
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [16]:
# 모델의 데이터 확인
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(512, 14, 15001), dtype=float32, numpy=
array([[[-1.0224229e-05, -2.7752580e-04, -1.2825328e-04, ...,
          4.4761804e-05,  1.0248564e-04,  1.9457337e-04],
        [ 1.8985357e-04, -6.0376868e-04, -3.1143182e-04, ...,
         -1.9653777e-04, -5.8816066e-05,  1.8992188e-04],
        [ 1.8232486e-04, -8.4343180e-04, -6.2991597e-04, ...,
         -2.5223842e-04, -2.2866552e-04,  1.7806405e-04],
        ...,
        [ 1.6102778e-03, -3.4189556e-04, -2.7676011e-04, ...,
         -4.4959932e-04, -5.1134476e-04,  1.6483878e-04],
        [ 1.6681906e-03, -4.6135817e-04, -1.9318514e-04, ...,
         -6.6915312e-04, -7.2519149e-04,  4.3512366e-04],
        [ 1.7134207e-03, -5.9933978e-04, -1.2384290e-04, ...,
         -9.6014561e-04, -9.2771277e-04,  7.0822152e-04]],

       [[-1.0224229e-05, -2.7752580e-04, -1.2825328e-04, ...,
          4.4761804e-05,  1.0248564e-04,  1.9457337e-04],
        [ 5.6094257e-05, -1.4613013e-04, -2.5617867e-04, ...,
          8.5987231e-05, 

In [17]:
# 모델의 최종 출력 shape는 (256, 14, 15001)
# 256은 batch_size, 14는 squence_length, 15001은 단어의 갯수(Dense Layer 출력 차원 수)

model.summary() # sequence_length를 모르기 때문에 Output shape를 정확하게 모른다.

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3840256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  15376025  
Total params: 32,855,961
Trainable params: 32,855,961
Non-trainable params: 0
_________________________________________________________________


In [18]:
# 모델 학습
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
243/243 [==============================] - 61s 250ms/step - loss: 3.7809
Epoch 2/30
243/243 [==============================] - 61s 253ms/step - loss: 3.1885
Epoch 3/30
243/243 [==============================] - 62s 254ms/step - loss: 3.0280
Epoch 4/30
243/243 [==============================] - 62s 254ms/step - loss: 2.9146
Epoch 5/30
243/243 [==============================] - 62s 255ms/step - loss: 2.8233
Epoch 6/30
243/243 [==============================] - 62s 256ms/step - loss: 2.7440
Epoch 7/30
243/243 [==============================] - 62s 254ms/step - loss: 2.6723
Epoch 8/30
243/243 [==============================] - 62s 255ms/step - loss: 2.6068
Epoch 9/30
243/243 [==============================] - 62s 255ms/step - loss: 2.5443
Epoch 10/30
243/243 [==============================] - 62s 255ms/step - loss: 2.4858
Epoch 11/30
243/243 [==============================] - 62s 255ms/step - loss: 2.4307
Epoch 12/30
243/243 [==============================] - 62s 255ms/step - lo

In [19]:
# 단어 생성 함수
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어를 하나씩 생성
    while True:
        predict = model(test_tensor)  # 입력받은 문장
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 예측한 단어가 새로 생성된 단어 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙이기
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 최종 생성된 자연어 문장

In [24]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i m not gonna crack <end> '

# 회고록

- range(n)도 reverse() 함수가 먹힌다는 걸 오늘 알았다...
- 예시에 주어진 train data 갯수는 124960인걸 보면 총 데이터는 156200개인 것 같은데 아무리 전처리 단계에서 조건에 맞게 처리해도 168000개 정도가 나온다. 아무튼 일단 돌려본다.
- 문장의 길이가 최대 15라는 이야기는 <start>, <end>를 포함하여 15가 되어야 하는 것 같아서 tokenize했을 때 문장의 길이가 13 이하인 것만 corpus로 만들었다.
- 학습 회차 별 생성된 문장 input : `<start> i love`
    - 1회차 `'<start> i love you , i love you <end> '`
    - 2회차 `'<start> i love you , i m not gonna crack <end> '`
    - 3회차`'<start> i love you to be a shot , i m not a man <end> '`
    - 4회차 `'<start> i love you , i m not stunning , i m a fool <end> '`
- batch_size를 각각 256, 512, 1024로 늘려서 진행했는데, 1epoch당 걸리는 시간이 74s, 62s, 59s 정도로 batch_size 배수 만큼의 차이는 없었다. batch_size가 배로 늘어나면 걸리느 시간도 당연히 반으로 줄어들 것이라 생각했는데 오산이었다.
- 1회차는 tokenize 했을 때 length가 15 이하인 것을 train_data로 사용하였다.
- 2, 3, 4회차는 tokenize 했을 때 length가 13 이하인 것을 train_data로 사용하였다.
- 3회차는 2회차랑 동일한 데이터에 padding 을 post에서 pre로 변경하였다. RNN에서는 뒤에 padding을 넣는 것 보다 앞쪽에 padding을 넣어주는 쪽이 마지막 결과에 paddind이 미치는 영향이 적어지기 때문에 더 좋은 성능을 낼 수 있다고 알고있기 때문이다.
- 근데 실제로는 pre padding 쪽이 loss가 더 크게 나왔다. 확인해보니 이론상으로는 pre padding이 성능이 더 좋지만 실제로는 post padding쪽이 성능이 더 잘 나와서 post padding을 많이 쓴다고 한다.
- batch_size를 변경해서 pre padding을 한 번 더 돌려보았더니 같은 조건에서의 post padding 보다 loss가 높았고 문장도 부자연스러웠다. 앞으로는 post padding을 사용해야겠다.

## 추가 훈련 파일   
[lyricist-post_padding_256](./[E-06]lyricist-post_padding_256.ipynb)   
[lyricist-post_padding_512](./[E-06]lyricist-post_padding_512.ipynb)   
[lyricist-pre_padding_512](./[E-06]lyricist-pre_padding_512.ipynb)   
[lyricist-pre_padding_1024](./[E-06]lyricist-pre_padding_512.ipynb)